In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from PIL import Image
img = Image.open('full_duck.jpg')
img_array = np.array(img)/255.0

In [2]:
(w,h)=np.size(img)
print(w,h)
print(img_array.size)

5946 13816
246449808


In [3]:
duck_mode0=Image.open('testduck.jpg')  #duck model
duck_array = np.array(duck_mode0)/255.0
print(duck_mode0.size)
print(np.shape(duck_array))

(40, 44)
(44, 40, 3)


In [4]:
no_duck_mode0=Image.open('noducktest.jpg')#no duck model
no_duck_array = np.array(no_duck_mode0)/255.0
print(no_duck_mode0.size)
print(np.shape(no_duck_array))

(28, 32)
(32, 28, 3)


In [5]:
#duck_model mu
total=duck_array.shape[0]*duck_array.shape[1] #像素點總數
duck_x = 0
duck_y = 0
duck_z = 0
for j in range(duck_array.shape[0]):  #每個像素點的RGB總和
      for i in range(duck_array.shape[1]): 
           duck_x = duck_x + duck_array[j][i][0]
           duck_y = duck_y + duck_array[j][i][1]
           duck_z = duck_z + duck_array[j][i][2]

a= duck_x /total #RGB的平均(mu)
b= duck_y /total
c= duck_z /total

duck_mu=[a,b,c]
print(duck_mu)


#duck_model.jpg  sigma
covar_x=0

for j in range(duck_array.shape[0]): #每個像素點
     for i in range(duck_array.shape[1]): 
            x_reduce_mu   =  duck_array[j][i]-duck_mu
            x_reduce_mu_T =  np.transpose([x_reduce_mu])
            covar_x = covar_x + ( x_reduce_mu * x_reduce_mu_T )
            
print(covar_x /(total-1))


[0.9213101604278164, 0.9488658645276513, 0.9769741532976997]
[[0.00065542 0.00061881 0.00050382]
 [0.00061881 0.00066909 0.00056263]
 [0.00050382 0.00056263 0.00051321]]


In [6]:
#no duck_model mu
total_noduck = no_duck_array.shape[0]*no_duck_array.shape[1] #像素點總和

no_duck_x = 0
no_duck_y = 0
no_duck_z = 0

for j in range(no_duck_array.shape[0]):  
    for i in range(no_duck_array.shape[1]): 
        no_duck_x = no_duck_x + no_duck_array[j][i][0]
        no_duck_y = no_duck_y + no_duck_array[j][i][1]
        no_duck_z = no_duck_z + no_duck_array[j][i][2]

a1= no_duck_x /total_noduck
b1= no_duck_y /total_noduck
c1= no_duck_z /total_noduck

no_duck_mu=[a1,b1,c1] #mu
print(no_duck_mu)
#no duck_model.jpg  sigma
 
no_covar_x=0

for j in range(no_duck_array.shape[0]): #每個像數點 
    for i in range(no_duck_array.shape[1]): 
        x_reduce_mu_no   =  no_duck_array[j][i]-no_duck_mu
        x_reduce_mu_no_T =  np.transpose([x_reduce_mu_no])
        no_covar_x = no_covar_x + ( x_reduce_mu_no * x_reduce_mu_no_T )
            
print(no_covar_x /(total_noduck-1))

[0.4953387605042024, 0.5199886204481784, 0.5600446428571445]
[[0.00265662 0.00251849 0.00256384]
 [0.00251849 0.00241649 0.0024488 ]
 [0.00256384 0.0024488  0.0024859 ]]


In [17]:
import PIL 
im = PIL.Image.new(mode = "RGB", size = img.size )
import math
im_array= np.array(im) 

PI=math.pi
DET=np.linalg.det(covar_x)
INVERSE= np.linalg.inv(covar_x)
EXP=[]

DET1=np.linalg.det(no_covar_x)
INVERSE1= np.linalg.inv(no_covar_x)
EXP1=[]

for j in range(img_array.shape[0]):  
    for i in range(img_array.shape[1]): 
        a=img_array[j][i]-duck_mu  #duck_model 
        a1=np.transpose([img_array[j][i]-duck_mu]) 
        b=np.dot(a,INVERSE) 
        d=np.dot(b,a1)
        x=1/( ( ( (2*PI)**3 )*DET )**0.5 ) * math.e**((-0.5)*d )

        
        n=img_array[j][i]-no_duck_mu #no_duck_model 
        f=np.transpose([img_array[j][i]-no_duck_mu]) 
        g=np.dot(n,INVERSE1) 
        h=np.dot(g,f)
        y=1/( ( ( (2*PI)**3 )*DET1 )**0.5 ) * math.e**((-0.5)*h )

        if(x > y):
            im_array[j][i]=[255,255,255] #白色
        else:
            im_array[j][i]=[0,0,0] #黑色
                
 

In [18]:
im=Image.fromarray(im_array) # numpy->image
im.show() #顯示圖片
         